In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"  # to avoid memory fragmentation

from saris import sigmap
from sionna.channel import (
    cir_to_ofdm_channel,
    subcarrier_frequencies,
    time_lag_discrete_time_channel,
    cir_to_time_channel,
    time_to_ofdm_channel,
)
import tensorflow as tf
from saris.utils import utils
from typing import Tuple, Optional
import numpy as np
import math

In [2]:
def compute_rot_angle(pt1: list, pt2: list) -> Tuple[float, float, float]:
    """Compute the rotation angles for vector pt1 to pt2."""
    x = pt2[0] - pt1[0]
    y = pt2[1] - pt1[1]
    z = pt2[2] - pt1[2]

    return cartesian2spherical(x, y, z)


def cartesian2spherical(x: float, y: float, z: float) -> Tuple[float, float, float]:
    r = math.sqrt(x**2 + y**2 + z**2)
    theta = math.atan2(y, x)
    phi = math.acos(z / r)
    return r, theta, phi


def spherical2cartesian(r: float, theta: float, phi: float) -> Tuple[float, float, float]:
    x = r * math.sin(phi) * math.cos(theta)
    y = r * math.sin(phi) * math.sin(theta)
    z = r * math.cos(phi)
    return x, y, z


compute_scene_path = "/home/hieule/research/saris/local_assets/blender/hallway_L_0/ceiling_idx/hallway.xml"
viz_scene_path = "/home/hieule/research/saris/local_assets/blender/hallway_L_0/idx/hallway.xml"
sionna_config = utils.load_config("/home/hieule/research/saris/configs/sionna_L_multi_users.yaml")

ris_pos = sionna_config["ris_positions"][0]
tx_pos = sionna_config["tx_positions"][0]
r, theta, phi = compute_rot_angle(tx_pos, ris_pos)
sionna_config["tx_orientations"] = [[theta, math.pi / 2 - phi, 0.0]]

In [3]:
sig_cmap = sigmap.engine.SignalCoverageMap(
        sionna_config, compute_scene_path, viz_scene_path
    )
bandwidth = 20e6
paths = sig_cmap.compute_paths()


In [4]:
paths.normalize_delays = False
cir = paths.cir()
# a: [batch_size, num_rx, num_rx_ant, num_tx, num_tx_ant, max_num_paths, num_time_steps], tf.complex
a, tau = cir
maximum_delay_spread=1e-6 # 1us
(l_min, l_max) = time_lag_discrete_time_channel(bandwidth, maximum_delay_spread)
print(f"l_min: {l_min}, l_max: {l_max}")
# [batch size, num_rx, num_rx_ant, num_tx, num_tx_ant, num_time_steps, l_max - l_min + 1], tf.complex
channels: tf.Tensor = cir_to_time_channel(bandwidth, a, tau, l_min, l_max)
channels.shape

l_min: -6, l_max: 26


TensorShape([1, 3, 2, 1, 16, 1, 33])

In [6]:
real_dtype = tau.dtype
c = tf.reduce_mean(tf.reduce_sum(tf.square(tf.abs(channels)),
                                         axis=6, keepdims=True),
                           axis=(2,4,5), keepdims=True)
path_gains = tf.squeeze(c, axis=(0, 2, 3, 4, 5, 6)).numpy()
c = tf.complex(tf.sqrt(c), tf.constant(0., real_dtype))
print(f"c: {c}; path_gains: {path_gains}")

c: [[[[[[[1.6809479e-05+0.j]]]]]




  [[[[[4.9246821e-05+0.j]]]]]




  [[[[[9.5863375e-05+0.j]]]]]]]; path_gains: [2.8255859e-10 2.4252496e-09 9.1897867e-09]


In [27]:
10*math.log10(1.6784123e-05)

-47.7510134647502

In [7]:
# [num_rx, num_rx_ant, num_tx, num_tx_ant, num_time_steps], tf.complex
h_time_sum_power = tf.reduce_sum(tf.abs(channels[0]) ** 2, axis=-1)
# [num_rx, num_rx_ant]
h_time_avg_power = tf.reduce_mean(h_time_sum_power, axis=(1,2, 3, 4))
h_time_avg_power

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([2.8255859e-10, 2.4252496e-09, 9.1897867e-09], dtype=float32)>

In [7]:
channels.shape

TensorShape([1, 3, 2, 1, 16, 1, 73])

In [8]:
channels_normalized_1 = channels / tf.sqrt(h_time_avg_power[None, ..., None, None, None, None, None])

TypeError: `x` and `y` must have the same dtype, got tf.complex64 != tf.float32.

In [9]:
channels_normalized = cir_to_time_channel(bandwidth, a, tau, l_min, l_max, normalize=True)

In [18]:
channels_normalized
h_time_sum_power = tf.reduce_sum(tf.abs(channels_normalized[0]) ** 2, axis=-1)
# [num_rx, num_rx_ant]
h_time_avg_power = tf.reduce_mean(h_time_sum_power, axis=(2, 3, 4))
h_time_avg_power

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[1.1298661 , 0.8701341 ],
       [1.1529571 , 0.84704286],
       [1.1243306 , 0.8756695 ]], dtype=float32)>